In [63]:
!head ../../../Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40


In [64]:
from pathlib import Path
import findspark

findspark.init(
    spark_home=str(Path.cwd() / ".." / "ext" / "spark-3.4.0-bin-hadoop3")
)

from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, max

# Flight data analysis (SQL)

In [65]:
# Spark UI is available at localhost:4040
spark = (
    SparkSession.builder.master("local")
    .appName("e1:flights")
    .getOrCreate()
)

In [66]:
# load data into data frame
flight_data = (
    spark.read.option("inferSchema", "true")
    .option("header", "true")
    .csv(
        "../../../Spark-The-Definitive-Guide/data/flight-data/csv/2015-summary.csv"
    )
)

In [67]:
flight_data.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [68]:
flight_data.show(3)
# flight_data.rdd.take(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
+-----------------+-------------------+-----+
only showing top 3 rows



In [69]:
# load data into a table
flight_data.createOrReplaceTempView("flight_data")

In [70]:
sql_way = spark.sql(
"""
SELECT DEST_COUNTRY_NAME, count(*)
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
ORDER BY 2 DESC
"""
)

# sql_way.explain()
# sql_way.show(3)

In [71]:
data_frame_way = flight_data\
    .groupBy("DEST_COUNTRY_NAME").count()\
    .sort(desc("count"))

# data_frame_way.explain()
# data_frame_way.show(3)

### Establish the maximum number of flights to and from any given location

In [72]:
# sql syntax
spark.sql("SELECT max(count) from flight_data").show(1)

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [73]:
# dataframe syntax
flight_data.select(max("count")).show(1)

+----------+
|max(count)|
+----------+
|    370002|
+----------+



### Find the top five destination countries in the data

In [74]:
# sql syntax
max_sql = spark.sql(
"""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
"""
)

max_sql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [75]:
# dataframe syntax

flight_data.groupBy("DEST_COUNTRY_NAME").sum("count").withColumnRenamed(
    "sum(count)", "destination_total"
).sort(desc("destination_total")).limit(5).show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [76]:
# close session
spark.stop()

## Spark SQL

In [77]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


with SparkSession.builder.getOrCreate() as sc:
    # Setup the Schema
    schema = StructType(
        [
            StructField("User ID", IntegerType(), True),
            StructField("Username", StringType(), True),
            StructField("Browser", StringType(), True),
            StructField("OS", StringType(), True),
        ]
    )

    # Add Data
    data = [
        (1580, "Barry", "FireFox", "Windows"),
        (5820, "Sam", "MS Edge", "Linux"),
        (2340, "Harry", "Vivaldi", "Windows"),
        (7860, "Albert", "Chrome", "Windows"),
        (1123, "May", "Safari", "macOS"),
    ]

    user_data_df = sc.createDataFrame(data, schema=schema)
    user_data_df.show()

+-------+--------+-------+-------+
|User ID|Username|Browser|     OS|
+-------+--------+-------+-------+
|   1580|   Barry|FireFox|Windows|
|   5820|     Sam|MS Edge|  Linux|
|   2340|   Harry|Vivaldi|Windows|
|   7860|  Albert| Chrome|Windows|
|   1123|     May| Safari|  macOS|
+-------+--------+-------+-------+

